In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import seaborn as sns
from imblearn.over_sampling import SMOTE
import gdown
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
arquivo_destino_base = "dataset_{}.csv"

ids = {
    "consumo_2024": "1-iXT7eaJWQokHf9cyfrB8N5wvkdhgjJW",
    "consumo_2023": "1-WfvkRwaRr85B_Joxcm9xVdpyg5NBAmp",
    "consumo_2022": "1-Uu4Tf4lufJVFeJnYKc5w7OeW66pe1eC",
    "consumo_2021": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
    "consumo_2020": "1-1pOoa0eJlNJ94BMi7p4PTx5KUS96mhX",
    "consumo_2019": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC"
}


dataframes = {}


for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)

In [ ]:
# juntando todos os dataframes em um só

arquivos_csv = [
    "./dataset_consumo_2024.csv",
    "./dataset_consumo_2023.csv",
    "./dataset_consumo_2022.csv",
    "./dataset_consumo_2021.csv",
    "./dataset_consumo_2019.csv"
]

ALL_COLUMNS_CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)

In [ ]:
file_id_fraudes = "1-MbIlChqQapcxFkoJgpbQIsN9FBLfbX1"
url_fraudes = f"https://drive.google.com/uc?id={file_id_fraudes}"

gdown.download(url_fraudes, quiet=False)

In [ ]:
# # diminuir dataset, reduzindo o numero de matrículas

# def filtrar_por_matriculas(df, n=1000):
#     matriculas = df['MATRICULA'].unique()[:n]

#     df_filtrado = df[df['MATRICULA'].isin(matriculas)]

#     return df_filtrado


# df = filtrar_por_matriculas(ALL_COLUMNS_CONSUMO_GERAL)

In [ ]:
fraudes = pd.read_csv('/content/fraudes.csv')

In [ ]:
# removendo colunas que não seram utilizandas

ALL_COLUMNS_CONSUMO_GERAL = ALL_COLUMNS_CONSUMO_GERAL.drop(columns=['Unnamed: 0', "SUB_CATEGORIA", "STA_TROCA", "STA_ACEITA_LEITURA", 'STA_TROCA', 'EMP_CODIGO', 'COD_GRUPO', 'COD_SETOR_COMERCIAL', 'NUM_QUADRA', 'COD_ROTA_LEITURA', 'SEQ_RESPONSAVEL', 'ECO_RESIDENCIAL', 'ECO_COMERCIAL', 'ECO_INDUSTRIAL', 'ECO_PUBLICA', 'ECO_OUTRAS','LTR_ATUAL', 'LTR_COLETADA', 'DAT_LEITURA', 'DIAS_LEITURA', 'COD_LEITURA_INF_1', 'COD_LEITURA_INF_2', 'COD_LEITURA_INF_3', 'HORA_LEITURA', 'DSC_SIMULTANEA', 'COD_LEITURA_INT','EXCECAO'])

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL.head()

In [ ]:
# remove linhas duplicadas

def verificar_duplicatas_referencia(df):
    df_duplicados = df[df.duplicated(subset=['MATRICULA', 'REFERENCIA'], keep=False)]

    return df_duplicados

df_sem_duplicadas = verificar_duplicatas_referencia(ALL_COLUMNS_CONSUMO_GERAL)

In [ ]:
sem_na = df_sem_duplicadas.dropna()

In [ ]:
def remove_rows_with_column_value_greater_than_one(df, column_names):
    # Verifica se algum valor em qualquer uma das colunas especificadas é maior que 1
    condition = (df[column_names] > -10).any(axis=1)

    # Filtra o DataFrame, removendo as linhas onde a condição é True
    df_filtered = df[~condition]

    return df_filtered

# Exemplo de uso:
df_sem_linhas = remove_rows_with_column_value_greater_than_one(sem_na, ['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
# tratando a longetude e latitude
# verificando a quantidade de clusters é mais apropriada utilizando K-Means


def plotar(n_clusters):
  #clusterizando
  df_temp = df_sem_linhas.copy()
  kmeans = KMeans(n_clusters=n_clusters, random_state=42)
  df_temp['cluster'] = kmeans.fit_predict(df_sem_linhas[['COD_LATITUDE', 'COD_LONGITUDE']])

  # visualizando
  plt.figure(figsize=(10, 6))
  plt.scatter(df_sem_linhas['COD_LONGITUDE'], df_sem_linhas['COD_LATITUDE'], c=df_temp['cluster'], cmap='viridis', marker='o', s=100)
  plt.title(f"K-Means Clustering com {n_clusters} Clusters")
  plt.xlabel("Longitude")
  plt.ylabel("Latitude")
  plt.show()



# visulizar clusters do range de 2 para 17
# for x in range(2, 17):
#   print()
#   plotar(x)

In [ ]:
# clusterizando os dados de longetude e latitude

kmeans = KMeans(n_clusters=20, random_state=42)
df_sem_linhas['cluster'] = kmeans.fit_predict(df_sem_linhas[['COD_LATITUDE', 'COD_LONGITUDE']])

In [ ]:
# removendo colunas de longetude e latude

df_loc_tratado = df_sem_linhas.drop(columns=['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
# fazendo one hot encoding

df_loc_tratado_ohc = pd.get_dummies(df_loc_tratado, columns = ['cluster'], dtype=int)

In [ ]:
df_loc_tratado_ohc.columns

In [ ]:
df_loc_tratado_ohc = df_loc_tratado_ohc[df_loc_tratado_ohc["CATEGORIA"].isin(["RESIDENCIAL"])]
df_loc_tratado_ohc = df_loc_tratado_ohc.drop(columns="CATEGORIA")

In [ ]:
scaler = RobustScaler()
df_loc_tratado_ohc[['VOLUME_ESTIMADO_ACUM', 'VOLUME_ESTIMADO']] = scaler.fit_transform(df_loc_tratado_ohc[['VOLUME_ESTIMADO_ACUM', 'VOLUME_ESTIMADO']])

In [ ]:
# pivotando a tabela

pivoted_df = pd.pivot_table(
    df_loc_tratado_ohc,
    index='MATRICULA',
    columns='REFERENCIA',
    values=['CONS_MEDIDO', 'VOLUME_ESTIMADO'],
    aggfunc='sum'
)

pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]
pivoted_df = pivoted_df.reset_index()


# adicinando as outras colunas

other_columns = df_loc_tratado_ohc.drop(columns=['CONS_MEDIDO', 'VOLUME_ESTIMADO', 'REFERENCIA']).drop_duplicates()


pivoted_df = pd.merge(pivoted_df, other_columns, on='MATRICULA', how='left')


pivoted_df = pivoted_df.fillna(0)

In [ ]:
pivoted_df["DSC_OCORRENCIA"].value_counts()

In [ ]:
# balanciando e simplificando a coluna do tipo de ocorrencia para normal e não normal


result = pivoted_df.copy()

result['DSC_OCORRENCIA'] = pivoted_df['DSC_OCORRENCIA'].apply(lambda x: 1 if x == 'NORMAL' else 0)

result.head()

In [ ]:
result = result.drop(columns=['FATURADO_MEDIA'])

In [ ]:
result["TIPO_LIGACAO"] = result["TIPO_LIGACAO"].replace({"Hidrometrado": 0, "Consumo Fixo": 1})

In [ ]:
result["TIPO_LIGACAO"] = result["TIPO_LIGACAO"].astype(int)

In [ ]:
result["TIPO_LIGACAO"].value_counts()

In [ ]:
result.dtypes

In [ ]:
def remove_outliers_and_plot(df, column_name):
    # 1. Calcular os quartis
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)

    # 2. Calcular o IQR
    IQR = Q3 - Q1

    # 3. Determinar os limites superior e inferior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # 4. Filtrar os dados para remover os outliers
    df_filtrado = df[(df[column_name] >= limite_inferior) & (df[column_name] <= limite_superior)]

    # 5. Plotar o boxplot com os dados filtrados
    sns.boxplot(x=df_filtrado[column_name])
    plt.title(f'Boxplot de {column_name} (Sem Outliers)')
    plt.xlabel(column_name)
    plt.show()

    # Retornar o DataFrame filtrado
    return df_filtrado

# Exemplo de uso:
df_sem_outliers = remove_outliers_and_plot(result, 'VOLUME_ESTIMADO_ACUM')

In [ ]:
df_sem_outliers["MATRICULA"] = df_sem_outliers["MATRICULA"].astype(int)

In [ ]:
df_sem_outliers

In [ ]:
dataframe_fraudes_premissa = fraudes[['MATRICULA', 'DESCRICAO']].drop_duplicates()
dataframe_fraudes_premissa = pd.get_dummies(dataframe_fraudes_premissa, columns=['DESCRICAO'], dtype='int')

In [ ]:
dataframe_pj_premissa = pd.merge(df_sem_outliers, dataframe_fraudes_premissa, on='MATRICULA', how='left')

In [ ]:
dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'] = dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].fillna(0)

In [ ]:
dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].value_counts()

In [ ]:
dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'] = dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].astype(int)

In [ ]:
def balanciar(df):
  smote = SMOTE(random_state=42)
  X = df.drop('DESCRICAO_IRREGULARIDADE IDENTIFICADA', axis=1)
  y = df['DESCRICAO_IRREGULARIDADE IDENTIFICADA']
  X_res, y_res = smote.fit_resample(X, y)
  return pd.concat([X_res, y_res], axis=1)


dataframe_pj_premissa = balanciar(dataframe_pj_premissa)

In [ ]:
dataframe_pj_premissa

In [ ]:
np.unique(dataframe_pj_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].values, return_counts=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataframe_pj_premissa.to_parquet('/content/drive/Shareddrives/LeakSeeker/pre_processado_pf.parquet')